# ETL: автоматизация подготовки данных семинары

## Урок 1. Модели данных и нормализация таблиц. Схема «звезда»

### Задача:

 - Определите в какой нормальной форме данная таблица, приведите её ко 2 и 3 нормальным формам последовательно.

| EmployeeID | Name     | JobCode | Job           | Citycode | Homecity |
|--------------|----------|----------|----------------|-----------|----------|
| E001         | Alice    | J01      | Chef           | 26        | Moscow   |
| E001         | Alice    | J02      | Waiter         | 26        | Moscow   |
| E002         | Bob      | J02      | Waiter         | 56        | Perm     |
| E002         | Bob      | J03      | Bartender      | 56        | Perm     |
| E003         | Alice    | J01      | Chef           | 56        | Perm     |

In [1]:
import init_spark_env

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").appName("My Spark Application").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/30 15:39:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col

In [6]:
# DataFrame с исходными данными
data = [
    ("E001", "Alice", "J01", "Chef", "26", "Moscow"),
    ("E001", "Alice", "J02", "Waiter", "26", "Moscow"),
    ("E002", "Bob", "J02", "Waiter", "56", "Perm"),
    ("E002", "Bob", "J03", "Bartender", "56", "Perm"),
    ("E003", "Alice", "J01", "Chef", "56", "Perm")
]

columns = ["EmployeeID", "Name", "JobCode", "Job", "Citycode", "Homecity"]
df = spark.createDataFrame(data, schema=columns)

In [12]:
# Разделяем на таблицы Employees и Jobs
employees_df = df.select("EmployeeID", "Name", "Citycode", "Homecity").distinct()
jobs_df = df.select("EmployeeID", "JobCode", "Job").distinct()

In [13]:
# Переименуем колонки для соответствия нормализованной схеме
employees_df = employees_df.withColumnRenamed("Citycode", "HomeCityCode")

In [18]:
# Перейдем ко второй нормальной форме, для этого нужно создать отдельную таблицу для городов
# Таблица Cities:
# - Citycode
# - City
cities_df = df.select("Citycode", "Homecity").distinct().withColumnRenamed("Homecity", "City")

# Теперь есть три таблицы в 2НФ: Employees, Jobs и Cities
# Для 3НФ нужно убедиться, что все поля зависят только от первичного ключа
# В таблицах уже соблюдена 3НФ, так как все не ключевые поля зависят только от первичного ключа

In [19]:
# Вывод результата
employees_df.show()
jobs_df.show()
cities_df.show()

+----------+-----+------------+--------+
|EmployeeID| Name|HomeCityCode|Homecity|
+----------+-----+------------+--------+
|      E001|Alice|          26|  Moscow|
|      E002|  Bob|          56|    Perm|
|      E003|Alice|          56|    Perm|
+----------+-----+------------+--------+

+----------+-------+---------+
|EmployeeID|JobCode|      Job|
+----------+-------+---------+
|      E001|    J01|     Chef|
|      E001|    J02|   Waiter|
|      E002|    J02|   Waiter|
|      E002|    J03|Bartender|
|      E003|    J01|     Chef|
+----------+-------+---------+

+--------+------+
|Citycode|  City|
+--------+------+
|      26|Moscow|
|      56|  Perm|
+--------+------+



In [20]:
spark.stop()